In [1]:
import pandas as pd
import seaborn as sns
from pydataset import data
import numpy as np
import os

from sklearn.model_selection import train_test_split
import sklearn.preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay

import prepare as p
import model as m
import explore as e
import acquire as a
from env import get_db_url

#turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = a.get_comet_data()

In [ ]:
df.drop(columns=['DT','A1', 'A2', 'A3', 'n_obs_used', 'per_y','condition_code', 'ad'], inplace = True)

In [ ]:
df['tp_cal']=df['tp_cal'].str.split(pat='.').str[0]

In [ ]:
df['tp_cal']=df['tp_cal'].str.split(pat='-').str[0]

In [ ]:
df=df.drop(index=[0,1])

In [ ]:
df['tp_cal'] = df['tp_cal'].astype('int')

In [ ]:
df = pd.get_dummies(df, columns = ['two_body'], drop_first=False)

In [ ]:
df = df.set_index('full_name')

In [ ]:
df.rename({'class': 'orbit'}, axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['orbit'].value_counts()

In [ ]:
df.class_PAR.value_counts()

## Prep Complete

In [3]:
df = p.pre_processing(df)

In [ ]:
df.head()

# Split

In [4]:
train, validate, test = p.split_data(df)

In [5]:
train_X, validate_X, test_X, train_y, validate_y, test_y = p.final_split(df, 'orbit')

In [6]:
train.shape

(1243, 10)

In [7]:
validate.shape

(533, 10)

In [8]:
test.shape

(445, 10)

# Explore